In [16]:
import pandas as pd
import pdb
import os
import glob
import sys
import logging
sys.path.append('add-profiles')
from argoDatabase import argoDatabase
import addFunctions as af

In [17]:
from types import SimpleNamespace

args = SimpleNamespace()
args.logName = 'demoLog.log'
args.subset = 'trouble'
args.npes = 1
args.dbName = 'argo-express-test'
args.dbDumpThreshold = 200
args.qcThreshold = '1'
args.basinFilename = 'basinmask_01.nc'
args.addToDb = True
args.removeExisting = True
args.removeAddedFileNames = False
args.adjustedOnly = False 
args.minDate = None
args.maxDate = None


In [18]:
logFileName = args.logName
af.format_logger(logFileName, level=logging.WARNING)
logging.warning('Start of log file')
ad = argoDatabase(args.dbName, 'profiles',
                  qcThreshold=args.qcThreshold, 
                  dbDumpThreshold=args.dbDumpThreshold,
                  removeExisting=args.removeExisting,
                  basinFilename=args.basinFilename,
                  addToDb=args.addToDb,
                  removeAddedFileNames=args.removeAddedFileNames, 
                  adjustedOnly=args.adjustedOnly)

After initializeing logger and argoDatabase object, we get a list of files to add

In [23]:
ncFileDir = af.get_mirror_dir(args)
print(ncFileDir)

/home/tyler/Desktop/argo-database/troublesome-files


We get a list of dacs that will be added

In [24]:
dacs = af.get_dacs(args.subset)
print(dacs)

['aoml', 'coriolis', 'nmdis', 'kordi', 'meds', 'kma', 'bodc', 'csio', 'incois', 'jma', 'csiro']


Next we generate a table of all .nc files found in ncFileDir and dacs subdirectories

In [26]:
df = af.get_df_to_add(ncFileDir, dacs=dacs)
print(df.shape)
df.head()

(2750, 7)


,file,filename,profile,platform,prefix,dac,cat
183,/home/tyler/Desktop/argo-database/troublesome-...,SD1900722_001.nc,1900722_1,1900722,SD,aoml,S
193,/home/tyler/Desktop/argo-database/troublesome-...,SD1900722_002.nc,1900722_2,1900722,SD,aoml,S
195,/home/tyler/Desktop/argo-database/troublesome-...,SD1900722_003.nc,1900722_3,1900722,SD,aoml,S
192,/home/tyler/Desktop/argo-database/troublesome-...,SD1900722_004.nc,1900722_4,1900722,SD,aoml,S
190,/home/tyler/Desktop/argo-database/troublesome-...,SD1900722_005.nc,1900722_5,1900722,SD,aoml,S


Optional parameters in args can filter out more data, such as to only include S profiles, or Mixed, etc.

In [29]:
df = af.reduce_files(args, df)
print(df.shape)
df.head()

(2750, 7)


,file,filename,profile,platform,prefix,dac,cat
183,/home/tyler/Desktop/argo-database/troublesome-...,SD1900722_001.nc,1900722_1,1900722,SD,aoml,S
193,/home/tyler/Desktop/argo-database/troublesome-...,SD1900722_002.nc,1900722_2,1900722,SD,aoml,S
195,/home/tyler/Desktop/argo-database/troublesome-...,SD1900722_003.nc,1900722_3,1900722,SD,aoml,S
192,/home/tyler/Desktop/argo-database/troublesome-...,SD1900722_004.nc,1900722_4,1900722,SD,aoml,S
190,/home/tyler/Desktop/argo-database/troublesome-...,SD1900722_005.nc,1900722_5,1900722,SD,aoml,S


In [ ]:
af.run_parallel_process(ad, df.file.tolist(), ncFileDir, args.npes)

logging.warning('End of log file')